# 4 Generating the Bank for the MicroTasks

To generate the bank for the microtasks I will use an API for an LLM.
The elemens that I will need are:

1) Question
2) Braoad options (all 6 letters)
3) Shorter options (only 3)
4) The "Reading material"


Basically I will create the perfect prompt that will use the columns of the df_courses to generate the microstasks. 


In [20]:
from pathlib import Path
import pandas as pd
import openai
print(openai.__version__)

2.8.1


## 1 Load the data and filter for max 2 courses

In [21]:
# load the csv file about the courses forwhih we have to gen the tasks
silver = Path("../data_programmes_courses/silver")

df_courses_tasks = pd.read_csv(silver / "df_courses_tasks_silver.csv", encoding="utf-8-sig")
print("The shape of the courses tasks dataframe is:", df_courses_tasks.shape)

# keep only first two courses from each programme
df_courses_tasks = df_courses_tasks.groupby("programme_title").head(2).reset_index(drop=True)
print("After keeping only first two courses from each programme the shape is:", df_courses_tasks.shape)

The shape of the courses tasks dataframe is: (21, 21)
After keeping only first two courses from each programme the shape is: (14, 21)


## 2. Set up the client 

In [22]:
#!pip install --upgrade openai
import os
from openai import OpenAI


# Create the OpenAI client
def get_openai_client():
    """
    Read the API key from a local file and return a client.
    The secrets folder is gitignored so the key never reaches the repo.
    """
    key_path = Path("../data_bank_microtasks") / "api_key.txt"

    # Read the key and strip spaces and newlines
    api_key = key_path.read_text(encoding="utf8").strip()

    # Create the client using this key
    client = OpenAI(api_key=api_key)

    return client

# Create the client once
client = get_openai_client()

In [23]:
models = client.models.list()
for m in models.data:
    print(m.id)

tts-1
dall-e-2
tts-1-1106
gpt-4-turbo-2024-04-09
gpt-audio
gpt-4o-mini-tts
gpt-4-turbo
gpt-realtime
gpt-realtime-2025-08-28
gpt-4.1-mini
gpt-4.1-mini-2025-04-14
gpt-3.5-turbo-1106
gpt-4-0125-preview
davinci-002
gpt-4-turbo-preview
gpt-4-0613
gpt-4
gpt-4.1
gpt-5.1-chat-latest
gpt-4.1-2025-04-14
dall-e-3
gpt-4.1-nano
gpt-3.5-turbo-instruct-0914
whisper-1
o1-2024-12-17
gpt-4.1-nano-2025-04-14
gpt-3.5-turbo-16k
gpt-audio-2025-08-28
gpt-3.5-turbo-instruct
gpt-4o-2024-11-20
gpt-4o-2024-05-13
gpt-4o-mini-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-search-preview
omni-moderation-latest
o1-pro
o1-pro-2025-03-19
gpt-5.1-codex-mini
gpt-4o-search-preview-2025-03-11
text-embedding-ada-002
gpt-4o-2024-08-06
o1
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-mini-audio-preview
gpt-image-1-mini
gpt-5-mini
gpt-image-1
gpt-5-mini-2025-08-07
omni-moderation-2024-09-26
gpt-5
gpt-5-nano-2025-08-07
gpt-4o-audio-preview-2024-12-17
gpt-5-nano
gpt-5.1-2025-11-13
tts-1-hd-1106
tts-1-hd
gpt-3.5-turbo-0

## 3. Define the Prompt
The system prompt carries all project logic once. We reuse it for all courses.


CHANGE: based on the programme vectors such as A: best 2, B medium two an C lowest 2.

In [ ]:
import json

SYSTEM_PROMPT = """
You create a single multiple choice microtask that feels like a first step in a real course task.

You will receive one JSON object with these fields:
- course_code
- course_name
- course_objective
- course_content
- additional_information_teaching_methods
- method_of_assessment
- template_type   one of graph_choice, assumption_check, operational_definition, design_choice

General rules:
1. Pick one concrete concept or method from the text.
2. Write a short stimulus that sounds like a real situation for a first year student in this course.
3. Create one tiny learn bubble with three short bullet points:
   a. definition
   b. method reminder
   c. common pitfall
4. Create four options A, B, C, D. Only one is the best first step. Others are plausible but wrong first steps.
5. Keep language simple and concrete. Do not mention RIASEC in the text.

Template rules:
- graph_choice: stimulus names two variables and a goal. Options are graph types, one correct.
- assumption_check: stimulus names an analysis decision. Options are checks or tests to do first.
- operational_definition: stimulus names a vague construct. Options are observable measurements.
- design_choice: stimulus names a research or design goal. Options are study designs or data collection strategies.


Output format:
Return one JSON object with keys:
- stimulus: string
- tiny_learn: list of three short strings
- options: list of four option objects with keys "label" and "text"
- correct_option: label of the best option, one of "A","B","C","D"

Very important:
Reply with a single JSON object only.
Start your reply with { and end your reply with }.
Do not wrap the JSON in code fences.
Do not add any explanation, commentary, or text outside the JSON object.

Do not add any explanation outside the JSON object.
"""


## 4. Define the helpers functions

In [25]:

def truncate(text, max_chars=1200):
    """
    Simple helper to shorten long course texts.
    """
    if text is None:
        return ""
    s = str(text)
    if len(s) <= max_chars:
        return s
    return s[:max_chars]


def build_course_payload(row, template_type):
    """
    Prepare the JSON object that will be sent to the model for one course.
    """
    return {
        "course_code": str(row.get("code", "")),
        "course_name": str(row.get("course_name", "")),
        "course_objective": truncate(row.get("course_objective", ""), 1200),
        "course_content": truncate(row.get("course_content", ""), 1200),
        "additional_information_teaching_methods": truncate(
            row.get("additional_information_teaching_methods", ""), 800
        ),
        "method_of_assessment": truncate(row.get("method_of_assessment", ""), 800),
        "template_type": template_type,
    }


def safe_parse_json(raw_text):
    """
    Try to parse the model output as JSON.
    If that fails, try to extract the first {...} block.
    If that also fails, raise a clear error.
    """
    if raw_text is None:
        raise ValueError("Model returned no text at all")

    text = raw_text.strip()

    # If the model returned an empty string
    if not text:
        raise ValueError("Model returned an empty string, no JSON to parse")

    # First simple attempt
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pass

    # Second attempt, look for first and last curly brace
    start = text.find("{")
    end = text.rfind("}")
    if start == -1 or end == -1 or end <= start:
        print("Could not find any JSON object in the text. Here is a preview:")
        print(text[:400])
        raise ValueError("No JSON object found in model output")

    candidate = text[start : end + 1]

    try:
        return json.loads(candidate)
    except json.JSONDecodeError as e:
        print("Failed to parse this JSON candidate:")
        print(candidate[:400])
        raise e



## 5. API call function

In [26]:
def generate_microtask_for_row(row, template_type="graph_choice", model_name="gpt-4.1-mini"):
    """
    Call the Responses API for a single course row and a single template type.

    Returns a Python dict with the parsed microtask plus some metadata.
    """

    payload = build_course_payload(row, template_type)

    response = client.responses.create(
        model=model_name,
        input=json.dumps(payload),
        instructions=SYSTEM_PROMPT,
        max_output_tokens=500,
    )

    raw_text = response.output_text

    # Debug print for the first few characters so we know what came back
    print("RAW TEXT PREVIEW:")
    print(repr((raw_text or "")[:300]))
    print("END RAW TEXT PREVIEW\n")

    # If the model returned nothing at all, fail early with context
    if raw_text is None or not raw_text.strip():
        print("Full response object for debugging:")
        print(response)
        raise ValueError("Model returned no text, check model name and prompt")

    # Use the safe parser
    task = safe_parse_json(raw_text)

    task["course_code"] = str(row.get("code", ""))
    task["course_name"] = str(row.get("course_name", ""))
    task["template_type"] = template_type

    return task



In [27]:
test_row = df_courses_tasks.iloc[0]
test_task = generate_microtask_for_row(test_row, template_type="graph_choice")
print(json.dumps(test_task, indent=2, ensure_ascii=False))

RAW TEXT PREVIEW:
'{\n  "stimulus": "You are studying the relationship between the age of ancient coins (years since minting) and the amount of wear they display, with the goal to visualize this relationship clearly.",\n  "tiny_learn": [\n    "Scatter plots show relationships between two continuous variables.",\n    "Plot'
END RAW TEXT PREVIEW

{
  "stimulus": "You are studying the relationship between the age of ancient coins (years since minting) and the amount of wear they display, with the goal to visualize this relationship clearly.",
  "tiny_learn": [
    "Scatter plots show relationships between two continuous variables.",
    "Plot age on the x-axis and wear on the y-axis to see trends.",
    "Avoid bar charts as they do not show continuous data well."
  ],
  "options": [
    {
      "label": "A",
      "text": "Create a scatter plot with coin age on the x-axis and wear level on the y-axis."
    },
    {
      "label": "B",
      "text": "Make a bar chart showing average wear f

In [9]:
test_row = df_courses_tasks.iloc[0]

payload = build_course_payload(test_row, "graph_choice")

response = client.responses.create(
    model="gpt-4.1-mini",   # see note on model names below
    input=json.dumps(payload),
    instructions=SYSTEM_PROMPT,
    max_output_tokens=500,
)

raw_text = response.output_text
print("RAW TEXT START")
print(repr(raw_text[:500]))
print("RAW TEXT END")

RAW TEXT START
'{\n  "stimulus": "You want to explore how the age of ancient coins relates to the number of symbols engraved on them to understand changes over time.",\n  "tiny_learn": [\n    "Definition: A scatterplot displays data points to show relationships between two numeric variables.",\n    "Method reminder: Scatterplots help visualize if one variable changes with another, like age vs. number of symbols.",\n    "Common pitfall: Bar charts or pie charts are not suitable for showing relationships between numer'
RAW TEXT END


## 6.Loop over courses, pick templates, save tasks

Now build a simple loop that chooses a template for each course, calls the function, and collects results.

In [ ]:
import numpy as np
from tqdm import tqdm  # optional progress bar, pip install tqdm

TEMPLATE_TYPES = [
    "graph_choice",
    "assumption_check",
    "operational_definition",
    "design_choice",
]

def pick_template_type(row):
    """
    Simple template picker.

    Right now it just samples at random.
    Later you can use smarter rules based on the content.
    """
    return np.random.choice(TEMPLATE_TYPES)


microtasks = []

# Limit for a first run to avoid burning a lot of tokens
max_tasks = 10

for i, (_, row) in enumerate(tqdm(df_courses_tasks.iterrows(), total=len(df_courses_tasks))):
    if i >= max_tasks:
        break

    template_type = pick_template_type(row)

    try:
        task = generate_microtask_for_row(row, template_type=template_type)
        microtasks.append(task)
    except Exception as e:
        # Simple error handling so one bad course does not kill the whole run
        print(f"Problem on row {i} with course {row.get('code')}: {e}")


# Turn into a dataframe or save as json lines
tasks_df = pd.DataFrame(microtasks)
print(tasks_df.head())

tasks_df.to_json(
    "microtasks.jsonl",
    orient="records",
    lines=True,
    force_ascii=False,
)
